This notebook is meant to test the best model for our project Food Item Recognition from Images.

Please download the model from our github repository: https://github.com/ReemSleiman/CMPS261-Project.git

In the models folder, you will find several saved models. The best one is saved under name: "model_before_tuning.h5"

Download the model and save it in the same directory of this notebook.


Part 1: Data preprocessing 

In [114]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.utils import to_categorical

Assuming that the 'test' folder is within a 'data' folder and it contains subfolders named after the class index (1 to 9), and each subfolder contains images corresponding to that class, we'll load images from each subfolder, extract their labels based on the subfolder name, and preprocess them for testing

In [115]:
# Preprocess the raw test data with subfolders and one-hot encode labels
def preprocess_test_data(test_data_folder, img_size=(150, 150)):
    test_images = []
    test_labels = []

    # Loop through each class subfolder
    for class_name in os.listdir(test_data_folder):
        class_folder = os.path.join(test_data_folder, class_name)
        if os.path.isdir(class_folder):
            class_label = int(class_name) - 1  # Extract class label from folder name, and subtract 1 to make it zero-indexed

            # Loop through each image in the class subfolder
            for img_name in os.listdir(class_folder):
                img_path = os.path.join(class_folder, img_name)
                # Load and resize the image
                img = load_img(img_path, target_size=img_size)
                # Convert image to array and normalize pixel values
                img_array = img_to_array(img) / 255.0
                # Append image array and corresponding label to lists
                test_images.append(img_array)
                test_labels.append(class_label)

    # Convert test_labels to one-hot encoded format
    test_labels = to_categorical(test_labels, num_classes=9)

    return np.array(test_images), np.array(test_labels)

In [116]:
# Define the path to the raw test data folder
raw_test_data_folder = os.path.join(os.getcwd(), "data", "test")

In [117]:
# Preprocess the raw test data
test_images, test_labels = preprocess_test_data(raw_test_data_folder)

Part 2: Loading the model 

In [ ]:
# Load the trained model
from tensorflow.keras.models import load_model

model_path = "model_before_tuning.h5"
model = load_model(model_path)

In [119]:
# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

Part 3: Testing

In [120]:
# Evaluate the model on the preprocessed test data
test_loss, test_accuracy = model.evaluate(test_images, test_labels)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 91ms/step - accuracy: 0.8384 - loss: 0.4814
Test Loss: 0.6745206117630005, Test Accuracy: 0.7710220217704773


Part 4: Testing for an image from the internet

In [121]:
import requests
from PIL import Image
from io import BytesIO

In [122]:
# Function to preprocess a single image from the internet
def preprocess_image_from_url(image_url, img_size=(150, 150)):
    response = requests.get(image_url)
    image = Image.open(BytesIO(response.content))
    image = image.resize(img_size)
    img_array = np.array(image) / 255.0  # Normalize pixel values
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    return img_array

In [123]:
# URL of the image you want to test, replace with the desired image URL 

# Example: image url to a bottle
image_url = "https://i5.walmartimages.com/asr/83568193-2418-40f0-8367-4fd80481e2f8_1.fc5284020d9e9b9bcd45d777a9e62bb5.jpeg" 

In [124]:
# Preprocess the image
test_image = preprocess_image_from_url(image_url)

# Use the loaded model to make predictions on the preprocessed image
predictions = model.predict(test_image)
predicted_class = np.argmax(predictions[0])

# Map predicted class index to class label (assuming class labels are 1 to 9)
class_labels = ["bottle", "basket", "food", "cup", "jar", "can", "dish", "mug", "glass"]
predicted_label = class_labels[predicted_class]

print("Predicted Class:", predicted_label)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
Predicted Class: bottle
